In [55]:
import json
import time
import csv
import os

def process_vrp_file(file_path, result_folder):
    # Load the JSON input from a file
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Extract data from JSON
    distance_matrix = data['distance_matrix']
    time_matrix = data['time_matrices'][0]
    demands = data['demands']
    serve_time = data['serve_time']
    vehicle_capacities = data['vehicle_capacities']
    vehicle_max_weight = data['vehicle_capacities']
    fixed_cost = data['fixed_cost']
    cost_per_km = data['cost_per_km']
    contract_distance = data['contract_distance']
    vehicle_volume = data['vehicle_volume']
    cost_per_minute = data['cost_per_minute']
    variable_cost_per_minute = data['variable_cost_per_minute']
    contract_time = data['contract_time']
    vehicle_average_speed = data['vehicle_average_speed']
    order_volume = data['order_volume']
    vehicle_volumes = data['vehicle_volumes']
    vehicle_contract_code = data['vehicle_contract_code']
    vehicle_max_travel_distance = data['vehicle_max_travel_distance']
    vehicle_max_travel_time = data['vehicle_max_available_time']
    order_types = data['order_type']
    depot = data['depot']
    order_id=data['order_id']
   
    num_vehicles = data['num_vehicles']
    paired_orders = data['paired_order']
    paired_dict = {pair[0]: pair[1] for pair in paired_orders}
    print(paired_dict)
    start_time = time.time()

   


    def select_seed_customers_by_proximity(distance_matrix, num_vehicles):
        depot_index = 0
        distances = [(i, distance_matrix[depot_index][i]) for i in range(1, len(distance_matrix))]
        distances.sort(key=lambda x: x[1])
        return [distances[i][0] for i in range(min(num_vehicles, len(distances)))]

    def calculate_route_cost(route, vehicle_index, distance_matrix, time_matrix):
        total_distance = 0
        total_time = 0

        if len(route) > 0:  
            depot_index = 0
            total_distance += distance_matrix[depot_index][route[0]]  
            total_time += time_matrix[depot_index][route[0]]

        for i in range(len(route) - 1):
            total_distance += distance_matrix[route[i]][route[i + 1]]
            total_time += time_matrix[route[i]][route[i + 1]]
            total_time += serve_time[route[i]]

        if len(route) > 0:  
            total_distance += distance_matrix[route[-1]][depot_index]  
            total_time += time_matrix[route[-1]][depot_index]
            total_time += serve_time[route[-1]]

        excess_distance = max(0, total_distance - contract_distance[vehicle_index])
        variable_distance_cost = total_distance * cost_per_km[vehicle_index]

        if excess_distance > 0:
            variable_distance_cost += excess_distance * cost_per_km[vehicle_index]

        total_cost = (fixed_cost[vehicle_index] +
                      variable_distance_cost +
                      (total_time * cost_per_minute[vehicle_index]))

        return total_cost, total_distance, total_time
        




    def is_valid_route(route, demands, vehicle_capacities, order_volume, vehicle_index, vehicle_volume, time_matrix, vehicle_max_travel_time, serve_time, vehicle_max_weight, order_types, paired_orders):
        paired_dict = {pair[0]: pair[1] for pair in paired_orders}

        max_volume = vehicle_volume[vehicle_index]
        available_weight = vehicle_capacities[vehicle_index]
        current_time = 0
        pickup_weight = 0
        dropoff_weight = 0
        current_volume = 0
        visited_pickups = set()
        visited_dropoffs = set()

        i = 0
        while i < len(route):
            if route[i] == 0:  # Skip the depot
                i += 1
                continue

            order_index = route[i] - 1
            current_order_type = order_types[order_index]

            # Handle pickups
            if current_order_type == "pickup":
                if pickup_weight + demands[route[i]] > vehicle_capacities[vehicle_index]:
                    paired_dropoff = paired_dict.get(route[i])
                    if paired_dropoff and paired_dropoff not in visited_dropoffs:
                        print(f"Breached capacity, inserting paired dropoff {paired_dropoff} to free capacity")
                        route.insert(i + 1, paired_dropoff)
                        i += 1
                        continue
                    else:
                        print("Invalid route: Cannot handle capacity breach without paired dropoff.")
                        return False

                pickup_weight += demands[route[i]]
                current_volume += order_volume[route[i]]
                available_weight -= demands[route[i]]

                if pickup_weight > vehicle_capacities[vehicle_index] or available_weight < 0:
                    print("Invalid route: Capacity exceeded on pickup")
                    return False

                visited_pickups.add(route[i])

            # Handle drop-offs
            elif current_order_type == "dropoff":
                paired_pickup = next((key for key, value in paired_dict.items() if value == route[i]), None)
                if paired_pickup and paired_pickup not in visited_pickups:
                    print("Invalid route: Pickup not completed for dropoff")
                    return False

                if paired_pickup:
                    
                    pickup_weight -= demands[paired_pickup]
                    available_weight += demands[paired_pickup]
                    current_volume -= order_volume[paired_pickup]
                    print('pickup weight and drop weight ',pickup_weight,available_weight)
                visited_dropoffs.add(route[i])

                dropoff_weight += demands[route[i]]
                if dropoff_weight > vehicle_capacities[vehicle_index]:
                    print("Invalid route: Capacity exceeded on dropoff")
                    return False

            current_time += serve_time[route[i]]

            if current_time > vehicle_max_travel_time[vehicle_index]:
                print("Invalid route: Time exceeded")
                return False

            if i > 0:
                travel_time = time_matrix[route[i - 1]][route[i]]
                current_time += travel_time
                if current_time > vehicle_max_travel_time[vehicle_index]:
                    print("Invalid route: Time exceeded after travel")
                    return False

            i += 1

        if pickup_weight > vehicle_capacities[vehicle_index] or current_volume > max_volume:
            print("Invalid route: Final weight/volume exceeded")
            return False

        print("Route is valid")
        return True


    # def local_cheapest_insertion_with_paired_orders(distance_matrix, time_matrix, num_vehicles, demands, vehicle_capacities, order_volume, vehicle_max_travel_time, serve_time, vehicle_max_weight, vehicle_volume, order_types, paired_orders):
    #     routes = [[] for _ in range(num_vehicles)]
    #     unvisited_customers = set(range(1, len(demands)))
    #     seed_candidates = select_seed_customers_by_proximity(distance_matrix, num_vehicles)

    #     visited_customers = set()
    #     paired_dict = {pair[0]: pair[1] for pair in paired_orders}

    #     # Initialize routes with a seed customer for each vehicle
    #     for v in range(num_vehicles):
    #         for seed_customer in seed_candidates:
    #             seed_demand = demands[seed_customer]
    #             seed_volume = order_volume[seed_customer]

    #             if (seed_demand <= vehicle_capacities[v] and
    #                 seed_volume <= vehicle_volume[v]):
    #                 routes[v].append(seed_customer)
    #                 unvisited_customers.remove(seed_customer)
    #                 visited_customers.add(seed_customer)
    #                 break
                

    #     pickup_weight = [0] * num_vehicles
    #     available_weight = vehicle_capacities[:]
    #     current_volume = [0] * num_vehicles

    #     for v in range(num_vehicles):
    #         while unvisited_customers:
    #             best_insertion_distance = float('inf')
    #             best_insertion_position = None
    #             best_customer = None
    #             current_route = routes[v]

    #             for customer in unvisited_customers:
    #                 if customer in visited_customers:
    #                     continue

    #                 for position in range(len(current_route) + 1):
    #                     new_route = current_route[:position] + [customer] + current_route[position:]

    #                     new_pickup_weight = pickup_weight[v]
    #                     new_available_weight = available_weight[v]
    #                     new_current_volume = current_volume[v]

    #                     paired_dropoff = paired_dict.get(customer, None)
    #                     if paired_dropoff and paired_dropoff in unvisited_customers:
    #                         new_route_with_dropoff = new_route + [paired_dropoff]

    #                         new_pickup_weight -= demands[customer]
    #                         new_current_volume += order_volume[customer]
    #                         new_available_weight += demands[customer]

    #                         if is_valid_route(new_route_with_dropoff, demands, vehicle_capacities, order_volume, v, vehicle_volume, time_matrix, vehicle_max_travel_time, serve_time, vehicle_max_weight, order_types, paired_orders):
    #                             # Calculate the distance incurred by this insertion
    #                             insertion_distance = distance_matrix[current_route[position - 1]][customer] + distance_matrix[customer][paired_dropoff] + distance_matrix[paired_dropoff][current_route[position]] if position < len(current_route) else distance_matrix[current_route[position - 1]][customer]

    #                             if insertion_distance < best_insertion_distance:
    #                                 best_insertion_distance = insertion_distance
    #                                 best_insertion_position = position
    #                                 best_customer = customer

    #                             continue
                        
    #                     new_pickup_weight -= demands[customer]
    #                     new_current_volume += order_volume[customer]
    #                     new_available_weight -= demands[customer]

    #                     if is_valid_route(new_route, demands, vehicle_capacities, order_volume, v, vehicle_volume, time_matrix, vehicle_max_travel_time, serve_time, vehicle_max_weight, order_types, paired_orders):
    #                         insertion_distance = distance_matrix[current_route[position - 1]][customer] + distance_matrix[customer][current_route[position]] if position < len(current_route) else distance_matrix[current_route[position - 1]][customer]

    #                         if insertion_distance < best_insertion_distance:
    #                             best_insertion_distance = insertion_distance
    #                             best_insertion_position = position
    #                             best_customer = customer

    #             if best_customer is not None and best_insertion_position is not None:
    #                 print(f"Inserting customer {best_customer} at position {best_insertion_position} in vehicle {v}'s route.")
    #                 routes[v] = routes[v][:best_insertion_position] + [best_customer] + routes[v][best_insertion_position:]
    #                 unvisited_customers.remove(best_customer)
    #                 visited_customers.add(best_customer)

    #                 pickup_weight[v] = new_pickup_weight
    #                 available_weight[v] = new_available_weight
    #                 current_volume[v] = new_current_volume

    #                 if paired_dropoff:
    #                     paired_dropoff = paired_dict[best_customer]
    #                     print(f"Inserting paired dropoff {paired_dropoff} after {best_customer} in vehicle {v}'s route.")
    #                     routes[v].append(paired_dropoff)
                        
    #                     unvisited_customers.remove(paired_dropoff)
    #                     visited_customers.add(paired_dropoff)
    #             else:
    #                 print(f"No valid insertion found for vehicle {v}, terminating route construction.")
    #                 break

    #     return routes
    def local_cheapest_insertion_with_paired_orders(distance_matrix, time_matrix, num_vehicles, demands, vehicle_capacities, order_volume, vehicle_max_travel_time, serve_time, vehicle_max_weight, vehicle_volume, order_types, paired_orders):
        routes = [[] for _ in range(num_vehicles)]
        unvisited_customers = set(range(1, len(demands)))
        seed_candidates = select_seed_customers_by_proximity(distance_matrix, num_vehicles)

        visited_customers = set()
        paired_dict = {pair[0]: pair[1] for pair in paired_orders}

        # Initialize routes with a seed customer for each vehicle
        for v in range(num_vehicles):
            if not unvisited_customers:
                # No customers left to assign, vehicle route can remain empty
                print(f"No unvisited customers left for vehicle {v}.")
                continue
            
            seed_assigned = False
            for seed_customer in seed_candidates:
                if seed_customer in unvisited_customers:  # Ensure the seed is still unvisited
                    seed_demand = demands[seed_customer]
                    seed_volume = order_volume[seed_customer]

                    if (seed_demand <= vehicle_capacities[v] and
                            seed_volume <= vehicle_volume[v]):
                        routes[v].append(seed_customer)
                        unvisited_customers.remove(seed_customer)
                        visited_customers.add(seed_customer)
                        seed_assigned = True
                        print(f"Vehicle {v} starts with customer {seed_customer}")
                        break

            if not seed_assigned:
                # No valid seed customer found for this vehicle; route remains empty
                print(f"No valid seed customer assigned for vehicle {v}. Moving to next vehicle.")
                continue

        pickup_weight = [0] * num_vehicles
        available_weight = vehicle_capacities[:]
        current_volume = [0] * num_vehicles

        # Route construction for each vehicle
        for v in range(num_vehicles):
            while unvisited_customers:
                best_insertion_distance = float('inf')
                best_insertion_position = None
                best_customer = None
                current_route = routes[v]

                # Find the best customer to insert
                for customer in unvisited_customers:
                    if customer in visited_customers:
                        continue

                    for position in range(len(current_route) + 1):
                        new_route = current_route[:position] + [customer] + current_route[position:]

                        new_pickup_weight = pickup_weight[v]
                        new_available_weight = available_weight[v]
                        new_current_volume = current_volume[v]

                        paired_dropoff = paired_dict.get(customer, None)
                        if paired_dropoff and paired_dropoff in unvisited_customers:
                            new_route_with_dropoff = new_route + [paired_dropoff]

                            new_pickup_weight -= demands[customer]
                            new_current_volume += order_volume[customer]
                            new_available_weight += demands[customer]

                            if is_valid_route(new_route_with_dropoff, demands, vehicle_capacities, order_volume, v, vehicle_volume, time_matrix, vehicle_max_travel_time, serve_time, vehicle_max_weight, order_types, paired_orders):
                                insertion_distance = (
                                    distance_matrix[current_route[position - 1]][customer] +
                                    distance_matrix[customer][paired_dropoff] +
                                    distance_matrix[paired_dropoff][current_route[position]]
                                    if position < len(current_route) else
                                    distance_matrix[current_route[position - 1]][customer]
                                )

                                if insertion_distance < best_insertion_distance:
                                    best_insertion_distance = insertion_distance
                                    best_insertion_position = position
                                    best_customer = customer

                            continue

                        new_pickup_weight -= demands[customer]
                        new_current_volume += order_volume[customer]
                        new_available_weight -= demands[customer]

                        if is_valid_route(new_route, demands, vehicle_capacities, order_volume, v, vehicle_volume, time_matrix, vehicle_max_travel_time, serve_time, vehicle_max_weight, order_types, paired_orders):
                            insertion_distance = (
                                distance_matrix[current_route[position - 1]][customer] +
                                distance_matrix[customer][current_route[position]]
                                if position < len(current_route) else
                                distance_matrix[current_route[position - 1]][customer]
                            )

                            if insertion_distance < best_insertion_distance:
                                best_insertion_distance = insertion_distance
                                best_insertion_position = position
                                best_customer = customer

                if best_customer is not None and best_insertion_position is not None:
                    print(f"Inserting customer {best_customer} at position {best_insertion_position} in vehicle {v}'s route.")
                    routes[v] = routes[v][:best_insertion_position] + [best_customer] + routes[v][best_insertion_position:]
                    unvisited_customers.remove(best_customer)
                    visited_customers.add(best_customer)

                    pickup_weight[v] = new_pickup_weight
                    available_weight[v] = new_available_weight
                    current_volume[v] = new_current_volume

                    if paired_dropoff:
                        paired_dropoff = paired_dict[best_customer]
                        print(f"Inserting paired dropoff {paired_dropoff} after {best_customer} in vehicle {v}'s route.")
                        routes[v].append(paired_dropoff)
                        
                        unvisited_customers.remove(paired_dropoff)
                        visited_customers.add(paired_dropoff)
                else:
                    print(f"No valid insertion found for vehicle {v}, terminating route construction.")
                    break

        return routes


    # Example usage
    final_routes = local_cheapest_insertion_with_paired_orders(distance_matrix, time_matrix, num_vehicles, demands, vehicle_capacities, order_volume, vehicle_max_travel_time, serve_time, vehicle_max_weight, vehicle_volume, order_types, paired_orders)


    csv_data = [["Vehicle", "Route", "Route Total Orders", "Dropoff Orders", "Pickup Orders", "Route Total Weight (kg)", "Route Total Volume (cm³)", "Route Cost", "Vehicle Fixed Cost", "Route Distance Cost", "Route Time Cost", "Route Distance (km)", "Vehicle Contract Distance (km)", "Detour Distance (km)", "Vehicle Cost per Unit Distance", "Vehicle Variable Cost per Unit Distance", "Route Time (s)", "Vehicle Contract Time (s)", "Overtime (s)", "Vehicle Cost per Unit Time", "Vehicle Variable Cost per Unit Time", "Vehicle Max Travel Distance (km)", "Vehicle Max Available Time (s)", "Vehicle Max Weight (kg)", "Vehicle Max Volume (cm³)", "Vehicle Distance Utilization (%)", "Vehicle Time Utilization (%)", "Vehicle Weight Utilization (%)", "Vehicle Volume Utilization (%)"]]
    
    total_cost = 0
    total_distance = 0
    total_time = 0
    

    for i, route in enumerate(final_routes):
        if len(route) > 0:
            route_cost, route_distance, route_time = calculate_route_cost(route, i, distance_matrix, time_matrix)
            

            
            route_total_orders = sum(1 for j in route if j != 0)  # Exclude depot

            pickup_orders = sum(1 for j in route if order_types[j - 1] == 'pickup') if len(route) > 0 else 0
            dropoff_orders = sum(1 for j in route if order_types[j - 1] == 'dropoff') if len(route) > 0 else 0
            
            route_total_weight = sum(demands[j] for j in route if j != 0)
            route_total_volume = sum(order_volume[j] for j in route if j != 0)
            
            vehicle_distance_utilization = (route_distance / vehicle_max_travel_distance[i]) * 100
            vehicle_time_utilization = (route_time / vehicle_max_travel_time[i]) * 100
            vehicle_weight_utilization = (route_total_weight / vehicle_capacities[i]) * 100
            vehicle_volume_utilization = (route_total_volume / vehicle_volume[i]) * 100
            
            detour_distance = max(0, route_distance - contract_distance[i])
            overtime = max(0, route_time - contract_time[i])
            
            csv_data.append([
                vehicle_contract_code[i],
                " -> ".join(map(str, route)),
                route_total_orders,
                dropoff_orders,
                pickup_orders,
                route_total_weight,
                route_total_volume,
                route_cost,
                fixed_cost[i],
                cost_per_km[i]*route_distance,
                (route_time * cost_per_minute[i]),
                route_distance,
                contract_distance[i],
                detour_distance,
                cost_per_km[i],
                cost_per_km[i],
                route_time,
                contract_time[i],
                overtime,
                cost_per_minute[i],
                variable_cost_per_minute[i],
                vehicle_max_travel_distance[i],
                vehicle_max_travel_time[i],
                vehicle_max_weight[i],
                vehicle_volume[i],
                vehicle_distance_utilization,
                vehicle_time_utilization,
                vehicle_weight_utilization,
                vehicle_volume_utilization
            ])

            total_cost += route_cost
            total_distance += route_distance
            total_time += route_time


    # Save the CSV results in a specific folder
    output_dir = os.path.join(result_folder, os.path.basename(file_path).replace('.json', ''))
    os.makedirs(output_dir, exist_ok=True)

    csv_file_path = os.path.join(output_dir, 'results.csv')
    with open(csv_file_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(csv_data)

    end_time = time.time()


    print(f"Processed file: {file_path}")
    print(f"Total cost: {total_cost}")
    print(f"Total distance: {total_distance}")
    print(f"Total time: {total_time}")
    print(f"Time taken: {end_time - start_time} seconds")
    print(f"Results saved to: {csv_file_path}")


    
    end_time = time.time()
    duration = end_time - start_time

    # Prepare result output
    result = {
        "routes": final_routes,
        "duration": duration
    }

    result_file = os.path.join(result_folder, "result.json")
    with open(result_file, 'w') as f:
        json.dump(result, f, indent=4)

    print(f"Results saved to {result_file}")

if __name__ == "__main__":
    input_file = 'data1.json'  # Replace with your input file path
    result_folder = '/home/delhivery/Desktop/vrp_project/data'  # Replace with your result folder path

    if not os.path.exists(result_folder):
        os.makedirs(result_folder)

    process_vrp_file(input_file, result_folder)


{4: 7}
Vehicle 0 starts with customer 1
No valid seed customer assigned for vehicle 1. Moving to next vehicle.
No valid seed customer assigned for vehicle 2. Moving to next vehicle.
Route is valid
Route is valid
Route is valid
Route is valid
pickup weight and drop weight  0 800
Route is valid
pickup weight and drop weight  0 800
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Invalid route: Pickup not completed for dropoff
Invalid route: Pickup not completed for dropoff
Route is valid
Route is valid
Inserting customer 2 at position 1 in vehicle 0's route.
Route is valid
Route is valid
Route is valid
pickup weight and drop weight  0 800
Route is valid
pickup weight and drop weight  0 800
Route is valid
pickup weight and drop weight  0 800
Route is valid
Route is valid
Route is valid
Route is valid
Invalid route: Capacity exceeded on dropoff
Invalid route: Capacity exceeded on dropoff
Invalid route: Capacity exceeded on dropoff
Invalid route: Pickup not complet

In [66]:
import json
import time
import csv
import os

def process_vrp_file(file_path, result_folder):
    # Load the JSON input from a file
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Extract data from JSON
    distance_matrix = data['distance_matrix']
    time_matrix = data['time_matrices'][0]
    demands = data['demands']
    serve_time = data['serve_time']
    vehicle_capacities = data['vehicle_capacities']
    vehicle_max_weight = data['vehicle_capacities']
    fixed_cost = data['fixed_cost']
    cost_per_km = data['cost_per_km']
    contract_distance = data['contract_distance']
    vehicle_volume = data['vehicle_volume']
    cost_per_minute = data['cost_per_minute']
    variable_cost_per_minute = data['variable_cost_per_minute']
    contract_time = data['contract_time']
    vehicle_average_speed = data['vehicle_average_speed']
    order_volume = data['order_volume']
    vehicle_volumes = data['vehicle_volumes']
    vehicle_contract_code = data['vehicle_contract_code']
    vehicle_max_travel_distance = data['vehicle_max_travel_distance']
    vehicle_max_travel_time = data['vehicle_max_available_time']
    order_types = data['order_type']
    depot = data['depot']
    order_id=data['order_id']
   
    num_vehicles = data['num_vehicles']
    paired_orders = data['paired_order']
    paired_dict = {pair[0]: pair[1] for pair in paired_orders}

    
    paired_pickup_order=data['paired_pickup_orders']
    pickup_order={pair[0]:pair[1] for pair in paired_orders}

    start_time = time.time()

   


    def select_seed_customers_by_proximity(distance_matrix, num_vehicles):
        depot_index = 0
        distances = [(i, distance_matrix[depot_index][i]) for i in range(1, len(distance_matrix))]
        distances.sort(key=lambda x: x[1])
        return [distances[i][0] for i in range(min(num_vehicles, len(distances)))]

    def calculate_route_cost(route, vehicle_index, distance_matrix, time_matrix):
        total_distance = 0
        total_time = 0

        if len(route) > 0:  
            depot_index = 0
            total_distance += distance_matrix[depot_index][route[0]]  
            total_time += time_matrix[depot_index][route[0]]

        for i in range(len(route) - 1):
            total_distance += distance_matrix[route[i]][route[i + 1]]
            total_time += time_matrix[route[i]][route[i + 1]]
            total_time += serve_time[route[i]]

        if len(route) > 0:  
            total_distance += distance_matrix[route[-1]][depot_index]  
            total_time += time_matrix[route[-1]][depot_index]
            total_time += serve_time[route[-1]]

        excess_distance = max(0, total_distance - contract_distance[vehicle_index])
        variable_distance_cost = total_distance * cost_per_km[vehicle_index]

        if excess_distance > 0:
            variable_distance_cost += excess_distance * cost_per_km[vehicle_index]

        total_cost = (fixed_cost[vehicle_index] +
                      variable_distance_cost +
                      (total_time * cost_per_minute[vehicle_index]))

        return total_cost, total_distance, total_time
        




    def is_valid_route(route, demands, vehicle_capacities, order_volume, vehicle_index, vehicle_volume, time_matrix, vehicle_max_travel_time, serve_time, vehicle_max_weight, order_types, paired_orders):
        paired_dict = {pair[0]: pair[1] for pair in paired_orders}

        paired_p={pair[0]:pair[1] for pair in paired_pickup_orders}

        max_volume = vehicle_volume[vehicle_index]
        available_weight = vehicle_capacities[vehicle_index]
        current_time = 0
        pickup_weight = 0
        dropoff_weight = 0
        current_volume = 0
        visited_pickups = set()
        visited_dropoffs = set()

        i = 0
        while i < len(route):
            if route[i] == 0:  # Skip the depot
                i += 1
                continue

            order_index = route[i] - 1
            current_order_type = order_types[order_index]

            # Handle pickups
            if current_order_type == "pickup":
                if pickup_weight + demands[route[i]] > vehicle_capacities[vehicle_index]:
                    paired_dropoff = paired_dict.get(route[i])
                    paired_p1=paired_p.get(route[i])



                    if (paired_dropoff and paired_dropoff not in visited_dropoffs) or (paired_p1 and paired_p1 not in visited_dropoffs):
                        print(f"Breached capacity, inserting paired dropoff {paired_dropoff} to free capacity")
                        route.insert(i + 1, paired_dropoff)
                        i += 1
                        continue
                    else:
                        print("Invalid route: Cannot handle capacity breach without paired dropoff.")
                        return False

                pickup_weight += demands[route[i]]
                current_volume += order_volume[route[i]]
                available_weight -= demands[route[i]]

                if pickup_weight > vehicle_capacities[vehicle_index] or available_weight < 0:
                    print("Invalid route: Capacity exceeded on pickup")
                    return False

                visited_pickups.add(route[i])

            # Handle drop-offs
            elif current_order_type == "dropoff":
                paired_pickup = next((key for key, value in paired_dict.items() if value == route[i]), None)
                if paired_pickup and paired_pickup not in visited_pickups:
                    print("Invalid route: Pickup not completed for dropoff")
                    return False

                if paired_pickup:
                    
                    pickup_weight -= demands[paired_pickup]
                    available_weight += demands[paired_pickup]
                    current_volume -= order_volume[paired_pickup]
                    print('pickup weight and drop weight ',pickup_weight,available_weight)
                visited_dropoffs.add(route[i])

                dropoff_weight += demands[route[i]]
                if dropoff_weight > vehicle_capacities[vehicle_index]:
                    print("Invalid route: Capacity exceeded on dropoff")
                    return False

            current_time += serve_time[route[i]]

            if current_time > vehicle_max_travel_time[vehicle_index]:
                print("Invalid route: Time exceeded")
                return False

            if i > 0:
                travel_time = time_matrix[route[i - 1]][route[i]]
                current_time += travel_time
                if current_time > vehicle_max_travel_time[vehicle_index]:
                    print("Invalid route: Time exceeded after travel")
                    return False

            i += 1

        if pickup_weight > vehicle_capacities[vehicle_index] or current_volume > max_volume:
            print("Invalid route: Final weight/volume exceeded")
            return False

        print("Route is valid")
        return True

    def local_cheapest_insertion_with_paired_orders(distance_matrix, time_matrix, num_vehicles, demands, vehicle_capacities, order_volume, vehicle_max_travel_time, serve_time, vehicle_max_weight, vehicle_volume, order_types, paired_orders):
        routes = [[] for _ in range(num_vehicles)]
        unvisited_customers = set(range(1, len(demands)))
        seed_candidates = select_seed_customers_by_proximity(distance_matrix, num_vehicles)

        visited_customers = set()
        paired_dict = {pair[0]: pair[1] for pair in paired_orders}

        # Initialize routes with a seed customer for each vehicle
        for v in range(num_vehicles):
            if not unvisited_customers:
                # No customers left to assign, vehicle route can remain empty
                print(f"No unvisited customers left for vehicle {v}.")
                continue
            
            seed_assigned = False
            for seed_customer in seed_candidates:
                if seed_customer in unvisited_customers:  # Ensure the seed is still unvisited
                    seed_demand = demands[seed_customer]
                    seed_volume = order_volume[seed_customer]

                    if (seed_demand <= vehicle_capacities[v] and
                            seed_volume <= vehicle_volume[v]):
                        routes[v].append(seed_customer)
                        unvisited_customers.remove(seed_customer)
                        visited_customers.add(seed_customer)
                        seed_assigned = True
                        print(f"Vehicle {v} starts with customer {seed_customer}")
                        break

            if not seed_assigned:
                # No valid seed customer found for this vehicle; route remains empty
                print(f"No valid seed customer assigned for vehicle {v}. Moving to next vehicle.")
                continue

        pickup_weight = [0] * num_vehicles
        available_weight = vehicle_capacities[:]
        current_volume = [0] * num_vehicles

        # Route construction for each vehicle
        for v in range(num_vehicles):
            while unvisited_customers:
                best_insertion_distance = float('inf')
                best_insertion_position = None
                best_customer = None
                current_route = routes[v]

                # Find the best customer to insert
                for customer in unvisited_customers:
                    if customer in visited_customers:
                        continue

                    for position in range(len(current_route) + 1):
                        new_route = current_route[:position] + [customer] + current_route[position:]

                        new_pickup_weight = pickup_weight[v]
                        new_available_weight = available_weight[v]
                        new_current_volume = current_volume[v]

                        paired_dropoff = paired_dict.get(customer, None)
                        if paired_dropoff and paired_dropoff in unvisited_customers:
                            new_route_with_dropoff = new_route + [paired_dropoff]

                            new_pickup_weight -= demands[customer]
                            new_current_volume += order_volume[customer]
                            new_available_weight += demands[customer]

                            if is_valid_route(new_route_with_dropoff, demands, vehicle_capacities, order_volume, v, vehicle_volume, time_matrix, vehicle_max_travel_time, serve_time, vehicle_max_weight, order_types, paired_orders):
                                insertion_distance = (
                                    distance_matrix[current_route[position - 1]][customer] +
                                    distance_matrix[customer][paired_dropoff] +
                                    distance_matrix[paired_dropoff][current_route[position]]
                                    if position < len(current_route) else
                                    distance_matrix[current_route[position - 1]][customer]
                                )

                                if insertion_distance < best_insertion_distance:
                                    best_insertion_distance = insertion_distance
                                    best_insertion_position = position
                                    best_customer = customer

                            continue

                        new_pickup_weight -= demands[customer]
                        new_current_volume += order_volume[customer]
                        new_available_weight -= demands[customer]

                        if is_valid_route(new_route, demands, vehicle_capacities, order_volume, v, vehicle_volume, time_matrix, vehicle_max_travel_time, serve_time, vehicle_max_weight, order_types, paired_orders):
                            insertion_distance = (
                                distance_matrix[current_route[position - 1]][customer] +
                                distance_matrix[customer][current_route[position]]
                                if position < len(current_route) else
                                distance_matrix[current_route[position - 1]][customer]
                            )

                            if insertion_distance < best_insertion_distance:
                                best_insertion_distance = insertion_distance
                                best_insertion_position = position
                                best_customer = customer

                if best_customer is not None and best_insertion_position is not None:
                    print(f"Inserting customer {best_customer} at position {best_insertion_position} in vehicle {v}'s route.")
                    routes[v] = routes[v][:best_insertion_position] + [best_customer] + routes[v][best_insertion_position:]
                    unvisited_customers.remove(best_customer)
                    visited_customers.add(best_customer)

                    pickup_weight[v] = new_pickup_weight
                    available_weight[v] = new_available_weight
                    current_volume[v] = new_current_volume

                    if paired_dropoff:
                        paired_dropoff = paired_dict[best_customer]
                        print(f"Inserting paired dropoff {paired_dropoff} after {best_customer} in vehicle {v}'s route.")
                        routes[v].append(paired_dropoff)
                        
                        unvisited_customers.remove(paired_dropoff)
                        visited_customers.add(paired_dropoff)
                else:
                    print(f"No valid insertion found for vehicle {v}, terminating route construction.")
                    break

        return routes


    # Example usage
    final_routes = local_cheapest_insertion_with_paired_orders(distance_matrix, time_matrix, num_vehicles, demands, vehicle_capacities, order_volume, vehicle_max_travel_time, serve_time, vehicle_max_weight, vehicle_volume, order_types, paired_orders)


    csv_data = [["Vehicle", "Route", "Route Total Orders", "Dropoff Orders", "Pickup Orders", "Route Total Weight (kg)", "Route Total Volume (cm³)", "Route Cost", "Vehicle Fixed Cost", "Route Distance Cost", "Route Time Cost", "Route Distance (km)", "Vehicle Contract Distance (km)", "Detour Distance (km)", "Vehicle Cost per Unit Distance", "Vehicle Variable Cost per Unit Distance", "Route Time (s)", "Vehicle Contract Time (s)", "Overtime (s)", "Vehicle Cost per Unit Time", "Vehicle Variable Cost per Unit Time", "Vehicle Max Travel Distance (km)", "Vehicle Max Available Time (s)", "Vehicle Max Weight (kg)", "Vehicle Max Volume (cm³)", "Vehicle Distance Utilization (%)", "Vehicle Time Utilization (%)", "Vehicle Weight Utilization (%)", "Vehicle Volume Utilization (%)"]]
    
    total_cost = 0
    total_distance = 0
    total_time = 0
    

    for i, route in enumerate(final_routes):
        if len(route) > 0:
            route_cost, route_distance, route_time = calculate_route_cost(route, i, distance_matrix, time_matrix)
            

            
            route_total_orders = sum(1 for j in route if j != 0)  # Exclude depot

            pickup_orders = sum(1 for j in route if order_types[j - 1] == 'pickup') if len(route) > 0 else 0
            dropoff_orders = sum(1 for j in route if order_types[j - 1] == 'dropoff') if len(route) > 0 else 0
            
            route_total_weight = sum(demands[j] for j in route if j != 0)
            route_total_volume = sum(order_volume[j] for j in route if j != 0)
            
            vehicle_distance_utilization = (route_distance / vehicle_max_travel_distance[i]) * 100
            vehicle_time_utilization = (route_time / vehicle_max_travel_time[i]) * 100
            vehicle_weight_utilization = (route_total_weight / vehicle_capacities[i]) * 100
            vehicle_volume_utilization = (route_total_volume / vehicle_volume[i]) * 100
            
            detour_distance = max(0, route_distance - contract_distance[i])
            overtime = max(0, route_time - contract_time[i])
            
            csv_data.append([
                vehicle_contract_code[i],
                " -> ".join(map(str, route)),
                route_total_orders,
                dropoff_orders,
                pickup_orders,
                route_total_weight,
                route_total_volume,
                route_cost,
                fixed_cost[i],
                cost_per_km[i]*route_distance,
                (route_time * cost_per_minute[i]),
                route_distance,
                contract_distance[i],
                detour_distance,
                cost_per_km[i],
                cost_per_km[i],
                route_time,
                contract_time[i],
                overtime,
                cost_per_minute[i],
                variable_cost_per_minute[i],
                vehicle_max_travel_distance[i],
                vehicle_max_travel_time[i],
                vehicle_max_weight[i],
                vehicle_volume[i],
                vehicle_distance_utilization,
                vehicle_time_utilization,
                vehicle_weight_utilization,
                vehicle_volume_utilization
            ])

            total_cost += route_cost
            total_distance += route_distance
            total_time += route_time


    # Save the CSV results in a specific folder
    output_dir = os.path.join(result_folder, os.path.basename(file_path).replace('.json', ''))
    os.makedirs(output_dir, exist_ok=True)

    csv_file_path = os.path.join(output_dir, 'results.csv')
    with open(csv_file_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(csv_data)

    end_time = time.time()


    print(f"Processed file: {file_path}")
    print(f"Total cost: {total_cost}")
    print(f"Total distance: {total_distance}")
    print(f"Total time: {total_time}")
    print(f"Time taken: {end_time - start_time} seconds")
    print(f"Results saved to: {csv_file_path}")


    
    end_time = time.time()
    duration = end_time - start_time

    # Prepare result output
    result = {
        "routes": final_routes,
        "duration": duration
    }

    result_file = os.path.join(result_folder, "result.json")
    with open(result_file, 'w') as f:
        json.dump(result, f, indent=4)

    print(f"Results saved to {result_file}")

if __name__ == "__main__":
    input_file = 'data2.json'  # Replace with your input file path
    result_folder = '/home/delhivery/Desktop/vrp_project/data'  # Replace with your result folder path

    if not os.path.exists(result_folder):
        os.makedirs(result_folder)

    process_vrp_file(input_file, result_folder)


{1: 2, 4: 8}
Vehicle 0 starts with customer 2
No valid seed customer assigned for vehicle 1. Moving to next vehicle.
No valid seed customer assigned for vehicle 2. Moving to next vehicle.
Route is valid
Route is valid
Route is valid
Route is valid
pickup weight and drop weight  77 723
Route is valid
pickup weight and drop weight  77 723
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Invalid route: Pickup not completed for dropoff
Invalid route: Pickup not completed for dropoff
Inserting customer 6 at position 1 in vehicle 0's route.
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
pickup weight and drop weight  77 723
Route is valid
pickup weight and drop weight  77 723
Route is valid
pickup weight and drop weight  77 723
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Invalid route: Pickup not completed for dropoff
Invalid route: P

In [4]:
import json
import time
import csv
import os

def process_vrp_file(file_path, result_folder):
    # Load the JSON input from a file
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Extract data from JSON
    distance_matrix = data['distance_matrix']
    time_matrix = data['time_matrices'][0]
    demands = data['demands']
    serve_time = data['serve_time']
    vehicle_capacities = data['vehicle_capacities']
    vehicle_max_weight = data['vehicle_capacities']
    fixed_cost = data['fixed_cost']
    cost_per_km = data['cost_per_km']
    contract_distance = data['contract_distance']
    vehicle_volume = data['vehicle_volume']
    cost_per_minute = data['cost_per_minute']
    variable_cost_per_minute = data['variable_cost_per_minute']
    contract_time = data['contract_time']
    vehicle_average_speed = data['vehicle_average_speed']
    order_volume = data['order_volume']
    vehicle_volumes = data['vehicle_volumes']
    vehicle_contract_code = data['vehicle_contract_code']
    vehicle_max_travel_distance = data['vehicle_max_travel_distance']
    vehicle_max_travel_time = data['vehicle_max_available_time']
    order_types = data['order_type']
    depot = data['depot']
    order_id=data['order_id']
   
    num_vehicles = data['num_vehicles']
    paired_orders = data['paired_order']
    paired_dict = {pair[0]: pair[1] for pair in paired_orders}

    
    paired_pickup_orders=data['paired_pickup_orders']
    pickup_order={pair[0]:pair[1] for pair in paired_orders}

    start_time = time.time()

   


    def select_seed_customers_by_proximity(distance_matrix, num_vehicles):
        depot_index = 0
        distances = [(i, distance_matrix[depot_index][i]) for i in range(1, len(distance_matrix))]
        distances.sort(key=lambda x: x[1])
        return [distances[i][0] for i in range(min(num_vehicles, len(distances)))]

    def calculate_route_cost(route, vehicle_index, distance_matrix, time_matrix):
        total_distance = 0
        total_time = 0

        if len(route) > 0:  
            depot_index = 0
            total_distance += distance_matrix[depot_index][route[0]]  
            total_time += time_matrix[depot_index][route[0]]

        for i in range(len(route) - 1):
            total_distance += distance_matrix[route[i]][route[i + 1]]
            total_time += time_matrix[route[i]][route[i + 1]]
            total_time += serve_time[route[i]]

        if len(route) > 0:  
            total_distance += distance_matrix[route[-1]][depot_index]  
            total_time += time_matrix[route[-1]][depot_index]
            total_time += serve_time[route[-1]]

        excess_distance = max(0, total_distance - contract_distance[vehicle_index])
        variable_distance_cost = total_distance * cost_per_km[vehicle_index]

        if excess_distance > 0:
            variable_distance_cost += excess_distance * cost_per_km[vehicle_index]

        total_cost = (fixed_cost[vehicle_index] +
                      variable_distance_cost +
                      (total_time * cost_per_minute[vehicle_index]))

        return total_cost, total_distance, total_time
        




    def is_valid_route(route, demands, vehicle_capacities, order_volume, vehicle_index, vehicle_volume, time_matrix, vehicle_max_travel_time, serve_time, vehicle_max_weight, order_types, paired_orders, paired_pickup_orders):
        paired_dict = {pair[0]: pair[1] for pair in paired_orders}
        paired_pickup_dict = {pair[0]: pair[1] for pair in paired_pickup_orders}

        max_volume = vehicle_volume[vehicle_index]
        available_weight = vehicle_capacities[vehicle_index]
        current_time = 0
        pickup_weight = 0
        dropoff_weight = 0
        current_volume = 0
        visited_pickups = set()
        visited_dropoffs = set()

        i = 0
        while i < len(route):
            if route[i] == 0:  # Skip the depot
                i += 1
                continue

            order_index = route[i] - 1
            current_order_type = order_types[order_index]

            # Handle pickups
            if current_order_type == "pickup":
                paired_dropoff = paired_dict.get(route[i])
                paired_pickup = paired_pickup_dict.get(route[i])

                # If the pickup order has a paired pickup, it must be added immediately after
                if paired_pickup and paired_pickup not in visited_pickups:
                    if i + 1 >= len(route) or route[i + 1] != paired_pickup:
                        print(f"Invalid route: Paired pickup {paired_pickup} must follow {route[i]}.")
                        return False

                if pickup_weight + demands[route[i]] > vehicle_capacities[vehicle_index]:
                    if paired_dropoff and paired_dropoff not in visited_dropoffs:
                        print(f"Breached capacity, inserting paired dropoff {paired_dropoff} to free capacity")
                        route.insert(i + 1, paired_dropoff)
                        i += 1
                        continue
                    else:
                        print("Invalid route: Cannot handle capacity breach without paired dropoff.")
                        return False

                pickup_weight += demands[route[i]]
                current_volume += order_volume[route[i]]
                available_weight -= demands[route[i]]

                if pickup_weight > vehicle_capacities[vehicle_index] or available_weight < 0:
                    print("Invalid route: Capacity exceeded on pickup")
                    return False

                visited_pickups.add(route[i])

            # Handle drop-offs
            elif current_order_type == "dropoff":
                paired_pickup = next((key for key, value in paired_dict.items() if value == route[i]), None)
                if paired_pickup and paired_pickup not in visited_pickups:
                    print("Invalid route: Pickup not completed for dropoff")
                    return False

                if paired_pickup:
                    pickup_weight -= demands[paired_pickup]
                    available_weight += demands[paired_pickup]
                    current_volume -= order_volume[paired_pickup]
                    print('pickup weight and drop weight ', pickup_weight, available_weight)
                visited_dropoffs.add(route[i])

                dropoff_weight += demands[route[i]]
                if dropoff_weight > vehicle_capacities[vehicle_index]:
                    print("Invalid route: Capacity exceeded on dropoff")
                    return False

            current_time += serve_time[route[i]]

            if current_time > vehicle_max_travel_time[vehicle_index]:
                print("Invalid route: Time exceeded")
                return False

            if i > 0:
                travel_time = time_matrix[route[i - 1]][route[i]]
                current_time += travel_time
                if current_time > vehicle_max_travel_time[vehicle_index]:
                    print("Invalid route: Time exceeded after travel")
                    return False

            i += 1

        if pickup_weight > vehicle_capacities[vehicle_index] or current_volume > max_volume:
            print("Invalid route: Final weight/volume exceeded")
            return False

        print("Route is valid")
        return True

    def local_cheapest_insertion_with_paired_orders(distance_matrix, time_matrix, num_vehicles, demands, vehicle_capacities, order_volume, vehicle_max_travel_time, serve_time, vehicle_max_weight, vehicle_volume, order_types, paired_orders, paired_pickup_orders):
        routes = [[] for _ in range(num_vehicles)]
        unvisited_customers = set(range(1, len(demands)))
        seed_candidates = select_seed_customers_by_proximity(distance_matrix, num_vehicles)

        visited_customers = set()
        paired_dict = {pair[0]: pair[1] for pair in paired_orders}
        paired_pickup_dict = {pair[0]: pair[1] for pair in paired_pickup_orders}

        # Initialize routes with a seed customer for each vehicle
        for v in range(num_vehicles):
            if not unvisited_customers:
                print(f"No unvisited customers left for vehicle {v}.")
                continue
            
            seed_assigned = False
            for seed_customer in seed_candidates:
                if seed_customer in unvisited_customers:  # Ensure the seed is still unvisited
                    seed_demand = demands[seed_customer]
                    seed_volume = order_volume[seed_customer]

                    if (seed_demand <= vehicle_capacities[v] and seed_volume <= vehicle_volume[v]):
                        routes[v].append(seed_customer)
                        unvisited_customers.remove(seed_customer)
                        visited_customers.add(seed_customer)
                        seed_assigned = True
                        print(f"Vehicle {v} starts with customer {seed_customer}")
                        break

            if not seed_assigned:
                print(f"No valid seed customer assigned for vehicle {v}. Moving to next vehicle.")
                continue

        pickup_weight = [0] * num_vehicles
        available_weight = vehicle_capacities[:]
        current_volume = [0] * num_vehicles

        # Route construction for each vehicle
        for v in range(num_vehicles):
            while unvisited_customers:
                best_insertion_distance = float('inf')
                best_insertion_position = None
                best_customer = None
                current_route = routes[v]

                # Find the best customer to insert
                for customer in unvisited_customers:
                    if customer in visited_customers:
                        continue

                    for position in range(len(current_route) + 1):
                        new_route = current_route[:position] + [customer] + current_route[position:]

                        paired_dropoff = paired_dict.get(customer, None)
                        paired_pickup = paired_pickup_dict.get(customer, None)

                        # If the customer has a paired pickup, insert it immediately after
                        if paired_pickup and paired_pickup in unvisited_customers:
                            new_route.insert(position + 1, paired_pickup)

                        new_pickup_weight = pickup_weight[v] - demands[customer]
                        new_current_volume = current_volume[v] + order_volume[customer]
                        new_available_weight = available_weight[v] - demands[customer]

                        if is_valid_route(new_route, demands, vehicle_capacities, order_volume, v, vehicle_volume, time_matrix, vehicle_max_travel_time, serve_time, vehicle_max_weight, order_types, paired_orders, paired_pickup_orders):
                            insertion_distance = (
                                distance_matrix[current_route[position - 1]][customer] +
                                distance_matrix[customer][current_route[position]]
                                if position < len(current_route) else
                                distance_matrix[current_route[position - 1]][customer]
                            )

                            if insertion_distance < best_insertion_distance:
                                best_insertion_distance = insertion_distance
                                best_insertion_position = position
                                best_customer = customer

                if best_customer is not None and best_insertion_position is not None:
                    print(f"Inserting customer {best_customer} at position {best_insertion_position} in vehicle {v}'s route.")
                    routes[v] = routes[v][:best_insertion_position] + [best_customer] + routes[v][best_insertion_position:]
                    unvisited_customers.remove(best_customer)
                    visited_customers.add(best_customer)

                    pickup_weight[v] = new_pickup_weight
                    available_weight[v] = new_available_weight
                    current_volume[v] = new_current_volume

                    # Handle paired dropoff
                    if paired_dropoff:
                        print(f"Inserting paired dropoff {paired_dropoff} after {best_customer} in vehicle {v}'s route.")
                        routes[v].append(paired_dropoff)
                        unvisited_customers.remove(paired_dropoff)
                        visited_customers.add(paired_dropoff)
                else:
                    print(f"No valid insertion found for vehicle {v}, terminating route construction.")
                    break

        return routes



    # Example usage
    final_routes = local_cheapest_insertion_with_paired_orders(distance_matrix, time_matrix, num_vehicles, demands, vehicle_capacities, order_volume, vehicle_max_travel_time, serve_time, vehicle_max_weight, vehicle_volume, order_types, paired_orders,paired_pickup_orders)


    csv_data = [["Vehicle", "Route", "Route Total Orders", "Dropoff Orders", "Pickup Orders", "Route Total Weight (kg)", "Route Total Volume (cm³)", "Route Cost", "Vehicle Fixed Cost", "Route Distance Cost", "Route Time Cost", "Route Distance (km)", "Vehicle Contract Distance (km)", "Detour Distance (km)", "Vehicle Cost per Unit Distance", "Vehicle Variable Cost per Unit Distance", "Route Time (s)", "Vehicle Contract Time (s)", "Overtime (s)", "Vehicle Cost per Unit Time", "Vehicle Variable Cost per Unit Time", "Vehicle Max Travel Distance (km)", "Vehicle Max Available Time (s)", "Vehicle Max Weight (kg)", "Vehicle Max Volume (cm³)", "Vehicle Distance Utilization (%)", "Vehicle Time Utilization (%)", "Vehicle Weight Utilization (%)", "Vehicle Volume Utilization (%)"]]
    
    total_cost = 0
    total_distance = 0
    total_time = 0
    

    for i, route in enumerate(final_routes):
        if len(route) > 0:
            route_cost, route_distance, route_time = calculate_route_cost(route, i, distance_matrix, time_matrix)
            

            
            route_total_orders = sum(1 for j in route if j != 0)  # Exclude depot

            pickup_orders = sum(1 for j in route if order_types[j - 1] == 'pickup') if len(route) > 0 else 0
            dropoff_orders = sum(1 for j in route if order_types[j - 1] == 'dropoff') if len(route) > 0 else 0
            
            route_total_weight = sum(demands[j] for j in route if j != 0)
            route_total_volume = sum(order_volume[j] for j in route if j != 0)
            
            vehicle_distance_utilization = (route_distance / vehicle_max_travel_distance[i]) * 100
            vehicle_time_utilization = (route_time / vehicle_max_travel_time[i]) * 100
            vehicle_weight_utilization = (route_total_weight / vehicle_capacities[i]) * 100
            vehicle_volume_utilization = (route_total_volume / vehicle_volume[i]) * 100
            
            detour_distance = max(0, route_distance - contract_distance[i])
            overtime = max(0, route_time - contract_time[i])
            
            csv_data.append([
                vehicle_contract_code[i],
                " -> ".join(map(str, route)),
                route_total_orders,
                dropoff_orders,
                pickup_orders,
                route_total_weight,
                route_total_volume,
                route_cost,
                fixed_cost[i],
                cost_per_km[i]*route_distance,
                (route_time * cost_per_minute[i]),
                route_distance,
                contract_distance[i],
                detour_distance,
                cost_per_km[i],
                cost_per_km[i],
                route_time,
                contract_time[i],
                overtime,
                cost_per_minute[i],
                variable_cost_per_minute[i],
                vehicle_max_travel_distance[i],
                vehicle_max_travel_time[i],
                vehicle_max_weight[i],
                vehicle_volume[i],
                vehicle_distance_utilization,
                vehicle_time_utilization,
                vehicle_weight_utilization,
                vehicle_volume_utilization
            ])

            total_cost += route_cost
            total_distance += route_distance
            total_time += route_time


    # Save the CSV results in a specific folder
    output_dir = os.path.join(result_folder, os.path.basename(file_path).replace('.json', ''))
    os.makedirs(output_dir, exist_ok=True)

    csv_file_path = os.path.join(output_dir, 'results.csv')
    with open(csv_file_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(csv_data)

    end_time = time.time()


    print(f"Processed file: {file_path}")
    print(f"Total cost: {total_cost}")
    print(f"Total distance: {total_distance}")
    print(f"Total time: {total_time}")
    print(f"Time taken: {end_time - start_time} seconds")
    print(f"Results saved to: {csv_file_path}")


    
    end_time = time.time()
    duration = end_time - start_time

    # Prepare result output
    result = {
        "routes": final_routes,
        "duration": duration
    }

    result_file = os.path.join(result_folder, "result.json")
    with open(result_file, 'w') as f:
        json.dump(result, f, indent=4)

    print(f"Results saved to {result_file}")

if __name__ == "__main__":
    input_file = 'data2.json'  # Replace with your input file path
    result_folder = '/home/delhivery/Desktop/vrp_project/data'  # Replace with your result folder path

    if not os.path.exists(result_folder):
        os.makedirs(result_folder)

    process_vrp_file(input_file, result_folder)


Vehicle 0 starts with customer 1
No valid seed customer assigned for vehicle 1. Moving to next vehicle.
No valid seed customer assigned for vehicle 2. Moving to next vehicle.
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Invalid route: Pickup not completed for dropoff
Invalid route: Pickup not completed for dropoff
Route is valid
Route is valid
Inserting customer 2 at position 1 in vehicle 0's route.
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Route is valid
Invalid route: Capacity exceeded on dropoff
Invalid route: Capacity exceeded on dropoff
Invalid route: Capacity exceeded on dropoff
Invalid route: Pickup not completed for dropoff
Invalid route: Pickup not completed for dropoff
Invalid route: Pickup not completed for dropoff
Route is valid
Route is valid
Route is valid
Inserting customer 3 at position 2 i